In [30]:
import numpy as np
import math

In [32]:
def rotz(q):
    radq = np.radians(q)
    sin = np.sin(radq)
    cos = np.cos(radq)
    return np.array([[cos,-1*sin, 0], [sin,cos,0],[0,0,1]])

def roty(q):
    radq = np.radians(q)
    sin = np.sin(radq)
    cos = np.cos(radq)
    return np.array([[cos,0,sin], [0,1,0],[-1*sin,0,cos]])

def rotx(q):
    radq = np.radians(q)
    sin = np.sin(radq)
    cos = np.cos(radq)
    return np.array([[1,0,0], [0,cos,-1*sin],[0,sin,cos]])

In [33]:
def rot(k,theta):
    I = np.identity(3)
    khat = hat(k)
    khat2 = khat.dot(khat)
    return I + math.sin(theta)*khat + (1.0 - math.cos(theta))*khat2

def hat(k):
    """
    Returns a 3 x 3 cross product matrix for a 3 x 1 vector
    
             [  0 -k3  k2]
     khat =  [ k3   0 -k1]
             [-k2  k1   0]
    
    :type    k: numpy.array
    :param   k: 3 x 1 vector
    :rtype:  numpy.array
    :return: the 3 x 3 cross product matrix    
    """
    
    khat=np.zeros((3,3))
    khat[0,1]=-k[2]
    khat[0,2]=k[1]
    khat[1,0]=k[2]
    khat[1,2]=-k[0]
    khat[2,0]=-k[1]
    khat[2,1]=k[0]    
    return khat

In [34]:
def fwdkin(q):
    ex = np.array([[1],[0],[0]])
    ey = np.array([[0],[1],[0]])
    ez = np.array([[0],[0],[1]])

    l0 = 0.061 # base to servo 1
    l1 = 0.0435 # servo 1 to servo 2
    l2 = 0.08285 # servo 2 to servo 3
    l3 = 0.08285 # servo 3 to servo 4
    l4 = 0.07385 # servo 4 to servo 5
    l5 = 0.05457 # servo 5 to gripper
    
    P01 = ez * (l0+l1)
    P12 = np.zeros([3,1])
    P23 = ex * l2
    P34 = ez * -l3
    P45 = np.zeros([3,1])
    P5T = ex * -(l4+l5)
    
    R01 = rotz(q[0])
    R12 = roty(-q[1])
    R23 = roty(-q[2])
    R34 = roty(-q[3])
    R45 = rotx(-q[4])
    R5T = np.eye(3)

    Rot = R01 @ R12 @ R23 @ R34 @ R45 @ R5T

    Pot = P01 + R01 @( P12 + R12 @( P23 + R23 @( P34 + R34 @( P45 + R45@P5T))))

    return Rot, Pot
    

In [35]:
q_test = [90, 100, 43, 23, 134]
Rot, Pot = fwdkin(q_test)
print(Rot, Pot)

[[-5.94134778e-17  6.94658370e-01 -7.19339800e-01]
 [-9.70295726e-01  1.74024048e-01  1.68053070e-01]
 [ 2.41921896e-01  6.97972334e-01  6.74024048e-01]] [[9.80201177e-18]
 [1.60079000e-01]
 [2.21190665e-01]]


In [36]:
#Code to Move the Robotic Arm and then Measure the Angles


import time
from Arm_Lib import Arm_Device

Arm = Arm_Device()
time.sleep(.1)

def arm_clamp_block(enable):
    if enable == 0:
        Arm.Arm_serial_servo_write(6, 60, 400)
    else:
        Arm.Arm_serial_servo_write(6, 135, 400)
    time.sleep(.5)

    
def arm_move(p, s_time = 500):
    for i in range(5):
        id = i + 1
        if id == 5:
            time.sleep(.1)
            Arm.Arm_serial_servo_write(id, p[i], int(s_time*1.2))
        else :
            Arm.Arm_serial_servo_write(id, p[i], s_time)
        time.sleep(.01)
    time.sleep(s_time/1000)

def arm_move_up():
    Arm.Arm_serial_servo_write(2, 90, 1500)
    Arm.Arm_serial_servo_write(3, 90, 1500)
    Arm.Arm_serial_servo_write(4, 90, 1500)
    time.sleep(.1)

In [37]:
#Define Test Cases to Measure

test1 = [90, 90, 90, 90, 90, 90]
test2 = [80, 120, 40, 0, 90, 90]
test3 = [0, 60, 60, 180, 0, 90]
test4 = [100, 60, 50, 90, 0, 90]
test5 = [140, 120, 120, 90, 45, 90]

In [38]:
#For Each Test Case, Run the block of code to get the calculated Rot and Pot, then measure the actual Rot and Pot after the arm has moved.
Rot, Pot = fwdkin(test1)
print(Rot, Pot)

arm_move(test1, 1100)
arm_clamp_block(1)

[[-1.12481984e-32 -1.22464680e-16 -1.00000000e+00]
 [-1.83697020e-16 -1.00000000e+00  1.22464680e-16]
 [-1.00000000e+00  1.83697020e-16 -1.12481984e-32]] [[2.37640687e-33]
 [3.88096694e-17]
 [3.98620000e-01]]


In [39]:
Rot, Pot = fwdkin(test2)
print(Rot, Pot)

arm_move(test2, 1100)
arm_clamp_block(1)

[[-1.63175911e-01  5.93911746e-02 -9.84807753e-01]
 [-9.25416578e-01  3.36824089e-01  1.73648178e-01]
 [ 3.42020143e-01  9.39692621e-01 -5.75395780e-17]] [[0.01868223]
 [0.10595221]
 [0.21018151]]


In [40]:
Rot, Pot = fwdkin(test3)
print(Rot, Pot)

arm_move(test3, 1100)
arm_clamp_block(1)

[[ 0.5        0.         0.8660254]
 [ 0.         1.         0.       ]
 [-0.8660254  0.         0.5      ]] [[0.0489652 ]
 [0.        ]
 [0.32889019]]


In [41]:
Rot, Pot = fwdkin(test4)
print(Rot, Pot)

arm_move(test4, 1100)
arm_clamp_block(1)

[[ 0.16317591 -0.98480775 -0.05939117]
 [-0.92541658 -0.17364818  0.33682409]
 [-0.34202014  0.         -0.93969262]] [[-0.04166755]
 [ 0.23630842]
 [ 0.2485088 ]]


In [42]:
Rot, Pot = fwdkin(test5)
print(Rot, Pot)

arm_move(test5, 1100)
arm_clamp_block(1)

[[-0.66341395 -0.18368187 -0.72535709]
 [ 0.5566704  -0.76893496 -0.31441548]
 [-0.5        -0.61237244  0.61237244]] [[ 0.17189286]
 [-0.14423523]
 [ 0.2818852 ]]
